In [1]:
import pandas as pd
import folium
from geopy.geocoders import Nominatim
import time

In [2]:
# 1. 讀取你的 csv
df = pd.read_csv('top_100_route_2024.csv')

# 確認資料格式
df.head(5)

,city pair,city1,country1,region1,city2,country2,region2,flights per day
0,Jeju-Seoul,Jeju,South Korea,Asia/Pacific,Seoul,South Korea,Asia/Pacific,226
1,Melbourne-Sydney,Melbourne,Australia,Asia/Pacific,Sydney,Australia,Asia/Pacific,157
2,Rio de Janeiro-Sao Paulo,Rio de Janeiro,Brazil,South-Atlantic,Sao Paulo,Brazil,South-Atlantic,152
3,Sapporo-Tokyo,Sapporo,Japan,Asia/Pacific,Tokyo,Japan,Asia/Pacific,147
4,Fukuoka-Tokyo,Fukuoka,Japan,Asia/Pacific,Tokyo,Japan,Asia/Pacific,137


In [ ]:
# 2. 初始化 geocoder
geolocator = Nominatim(user_agent="geoapiExercises")

# 定義查緯度經度的函數
def get_lat_lon(city, country):
    try:
        location = geolocator.geocode(f"{city}, {country}")
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"找不到 {city}, {country}：{e}")
    return None, None

# 3. 對每一列城市去查經緯度
city1_lat, city1_lon, city2_lat, city2_lon = [], [], [], []

for idx, row in df.iterrows():
    lat1, lon1 = get_lat_lon(row['city1'], row['country1'])
    lat2, lon2 = get_lat_lon(row['city2'], row['country2'])
    
    city1_lat.append(lat1)
    city1_lon.append(lon1)
    city2_lat.append(lat2)
    city2_lon.append(lon2)

    time.sleep(1)  # 避免查太快被鎖IP，sleep 1秒

# 把查好的經緯度加回原本的df
df['city1_lat'] = city1_lat
df['city1_lon'] = city1_lon
df['city2_lat'] = city2_lat
df['city2_lon'] = city2_lon

# 4. 建一個世界地圖
m = folium.Map(location=[20, 0], zoom_start=2)

# 5. 畫每一條城市對城市的航線
for idx, row in df.iterrows():
    if pd.notnull(row['city1_lat']) and pd.notnull(row['city2_lat']):
        folium.PolyLine(
            locations=[(row['city1_lat'], row['city1_lon']), (row['city2_lat'], row['city2_lon'])],
            color='blue',
            weight=row['flights_per_day'] / 50,  # 根據每天航班數調整線粗
            opacity=0.7
        ).add_to(m)

# 6. 儲存地圖
m.save('top_100_routes_map.html')
print("✅ 地圖已儲存為 top_100_routes_map.html")


找不到 Jeju, South Korea：Non-successful status code 403
找不到 Seoul, South Korea：HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Seoul%2C+South+Korea&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
找不到 Melbourne, Australia：HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Melbourne%2C+Australia&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
找不到 Sydney, Australia：Non-successful status code 503
找不到 Rio de Janeiro, Brazil：Non-successful status code 503
找不到 Sao Paulo, Brazil：Non-successful status code 503
找不到 Sapporo, Japan：Non-successful status code 503
找不到 Tokyo, Japan：Non-successful status code 503
找不到 Fukuoka, Japan：Non-successful status code 503
找不到 Tokyo, Japan：Non-su